# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/jovyan/work


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [22]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# the total number of rows 
print('Total number of rows: ',len(full_data_rows_list))
# check to see what the list of event data rows will look like
print('Sample event data:')
print(full_data_rows_list[0:2])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

Total number of rows:  8056
Sample event data:
[['Harmonia', 'Logged In', 'Ryan', 'M', '0', 'Smith', '655.77751', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'Sehr kosmisch', '200', '1.54224E+12', '26'], ['The Prodigy', 'Logged In', 'Ryan', 'M', '1', 'Smith', '260.07465', 'free', 'San Jose-Sunnyvale-Santa Clara, CA', 'PUT', 'NextSong', '1.54102E+12', '583', 'The Big Gundown', '200', '1.54224E+12', '26']]
CPU times: user 84 ms, sys: 17.9 ms, total: 102 ms
Wall time: 192 ms


In [17]:
# check the number of rows in the csv file (including one header row)
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra database modelling 

Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, which was just created in Part I.  This *event_datafile_new.csv* contains the following columns: 

- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

Below is a sample of what the denormalized data look like in the <font color=red>**event_datafile_new.csv**</font>:

|    | artist      | firstName   | gender   |   itemInSession | lastName   |   length | level   | location                             |   sessionId | song                                           |   userId |
|---:|:------------|:------------|:---------|----------------:|:-----------|---------:|:--------|:-------------------------------------|------------:|:-----------------------------------------------|---------:|
|  0 | Harmonia    | Ryan        | M        |               0 | Smith      |  655.778 | free    | San Jose-Sunnyvale-Santa Clara, CA   |         583 | Sehr kosmisch                                  |       26 |
|  1 | The Prodigy | Ryan        | M        |               1 | Smith      |  260.075 | free    | San Jose-Sunnyvale-Santa Clara, CA   |         583 | The Big Gundown                                |       26 |
|  2 | Train       | Ryan        | M        |               2 | Smith      |  205.453 | free    | San Jose-Sunnyvale-Santa Clara, CA   |         583 | Marry Me                                       |       26 |
|  3 | Sony Wonder | Samuel      | M        |               0 | Gonzalez   |  218.07  | free    | Houston-The Woodlands-Sugar Land, TX |         597 | Blackbird                                      |       61 |
|  4 | Van Halen   | Tegan       | F        |               2 | Levine     |  289.384 | paid    | Portland-South Portland, ME          |         602 | Best Of Both Worlds (Remastered Album Version) |       80 |

## Connect to Appache Cassandra and Create Keyspace

*A python wrapper/ python driver called cassandra was used  to connect with Apache Cassandra and run queries.*

#### Creating a Cluster

In [35]:
# This should make a connection to a Cassandra instance in your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['mycassandra'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    print('Connection established!')
except Exception as e:
    print('Connection failed due to: ')
    print(e)



Connection established!


#### Create Keyspace

In [36]:
# Create a Keyspace

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [37]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Model data to the queries with Apache Cassandra

<div class="alert alert-block alert-info">
<b>#1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4.</b>
</div>


Since the question is looking for the specific `session_id` and `item_in_session`, these two columns can be used as Composite Partition Key. This should make each row unique. And in this case, the Partition Key is the Primary Key. 

In [38]:
# create_table

query = "CREATE TABLE IF NOT EXISTS song_info_by_sessionId_and_itemInSession "
query = query + "(session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY ((session_id, item_in_session)))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [40]:
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26


In [48]:
# insert values into table created above

file = 'event_datafile_new.csv'

# read lines in csv file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    # loop all the lines and insert into table.
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_by_sessionId_and_itemInSession (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]),line[0],line[9],float(line[5])))

In [49]:
## verify the data was entered into the table

query = "SELECT artist, song, length FROM song_info_by_sessionId_and_itemInSession WHERE session_id=338 AND item_in_session=4"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
    
for result in results:
    print(result)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


<div class="alert alert-block alert-info">
<b>#2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.</b>
</div>

Since the query is looking for the specific `user_id` and `session_id`, these two columns can be used as Composite Partition Key. From there, to make sure Primary Key is unique, Clustering Columns might be needed. In this case, song also need to be sorted by `item_in_session`, so  `item_in_session` could be used as Clustering Column to make the data unique. That should be enough to make the row Primary key unique.

In [50]:
# create_table

query = "CREATE TABLE IF NOT EXISTS song_and_user_info_by_userId_and_sessionId "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, PRIMARY KEY ((user_id,session_id),item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)     

In [51]:
# insert values into table created above

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_and_user_info_by_userId_and_sessionId (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(float(line[10])),int(line[8]), int(line[3]),line[0],line[9],line[1],line[4]))

In [52]:
# verify table with a query

query = "SELECT artist, song, first_name, last_name \
         FROM song_and_user_info_by_userId_and_sessionId \
         WHERE user_id=10 AND session_id=182 \
         ORDER BY item_in_session"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
    
for result in results:
    print(result)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


<div class="alert alert-block alert-info">
<b>#3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'.</b>
</div>

Since the query is looking for the specific `song`, the `song` can be used as Partition Key. From there, to make sure Primary Key is unique, Clustering Columns might be needed. In this case, `first_name` and `last_name` could be used as Clustering Column to make the data unique. That should be enough to make the row Primary key unique.

In [60]:
# create_table

query = "CREATE TABLE IF NOT EXISTS song_by_user "
query = query + "(song text, first_name text, last_name text, PRIMARY KEY (song,first_name,last_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [61]:
# insert values into table created above

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_by_user (song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9],line[1],line[4]))

In [62]:
# verify table with a query

query = "SELECT first_name, last_name FROM song_by_user WHERE song='All Hands Against His Own'"
try:
    results = session.execute(query)
except Exception as e:
    print(e)
    
for result in results:
    print(result)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Sara', last_name='Johnson')
Row(first_name='Tegan', last_name='Levine')


## Drop the tables before closing out the sessions

In [63]:
## Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS song_info_by_sessionId_and_itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
query = "DROP TABLE IF EXISTS song_and_user_info_by_userId_and_sessionId"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
query = "DROP TABLE IF EXISTS song_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [64]:
session.shutdown()
cluster.shutdown()